### SberHackaton
### team WhyNot
In that task we have a datasets with information about customers purchases and we need to predict a list of purshases that current customer will commit next

### 1. Download the datasets and observe

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from numpy.linalg import inv
import random
import pandas as pd

### Let's analyze orders dataset

In [2]:

orders = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_1_orders.csv')
city= pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/tab_6_city.csv')

orders.sample(5)

,user_id,order_id,order_created_time,retailer,store_id,platform
1702431,1967319,10786605,2020-05-26 07:42:30,Ашан,273,web
129341,243809,14164051,2020-07-29 11:44:29,METRO,103,app
1910935,2184543,12235325,2020-06-18 20:45:25,METRO,82,web
690233,1098590,10977234,2020-05-29 09:35:41,Ашан,272,app
1124501,1499924,17043849,2020-09-20 12:14:57,METRO,12,web


In [3]:
city.sample(5)

,store_id,city_name
573,682,Москва
590,672,Москва
95,260,Самара
74,183,Москва
79,99,Казань


In [4]:
city.shape

(788, 2)

In [5]:
orders.shape

(2461958, 6)

### Let's add information about the city where every order was made

In [6]:
orders_c = pd.merge(orders,city,on = 'store_id')
orders_c.sample(3)

,user_id,order_id,order_created_time,retailer,store_id,platform,city_name
893754,1643441,9023831,2020-04-25 12:19:31,Лента,166,web,Нижний Новгород
589669,1649169,16140666,2020-09-05 05:55:20,Лента,142,app,Волгоград
955998,1555694,8590379,2020-04-16 19:07:55,METRO,192,app,Ульяновск


### Here we can see different retailers in different cities, so if a person from Абакан there is only point to predict that he wil buy something in "Аллея"

In [7]:
city_ratailers = orders_c.groupby(['city_name'])['retailer'].value_counts()
city_ratailers

city_name           retailer            
 Каменск-Уральский  Лента                     49
Абакан              Аллея                    668
Альметьевск         Лента                   1904
Анадырь             ТЦ "Новомариинский"       46
Архангельск         METRO                   1962
                                            ... 
Якутск              УДАЧНАЯ ПОКУПКА          235
                    МаксМаркет               231
Ярославль           METRO                   9515
                    Лента                   7717
                    Ашан                    5305
Name: retailer, Length: 246, dtype: int64

### Let's calculate some additional info

In [8]:
print('number of users in the dataset : ' ,orders_c.user_id.nunique()) 

number of users in the dataset :  753572


### **We can see that number of users are less than number of purshases in the dataset so some customers made more than 1 purchase** 

### **Let's find users who made more purchases than others**

In [9]:
orders_per_customers = orders_c.groupby(['user_id'])['order_id'].count()
orders_per_customers

user_id
72         1
83         1
142        1
187        1
224        1
          ..
3432957    1
3432966    1
3432977    1
3433000    1
3433023    1
Name: order_id, Length: 753572, dtype: int64

In [10]:
orders_per_customers[orders_per_customers > 1]
#there are a lot of people who made more than one purshase

user_id
260         2
353         8
400         6
525         3
576        10
           ..
3424582     2
3427265     2
3427304     2
3432418     2
3432861     2
Name: order_id, Length: 355359, dtype: int64

In [11]:
orders_per_customers[orders_per_customers > 15] 
#here we have people who made a lot of purshases

user_id
766        25
1578       27
1724       25
2268       30
2693       18
           ..
2987260    26
3000696    19
3010057    23
3058685    52
3116617    16
Name: order_id, Length: 24406, dtype: int64

### We can see that there are a lot of people who made a lot (more than 15) purchases


### There is a list of people who made more purhases that others

In [12]:
orders_per_customers[orders_per_customers > 100] 

user_id
144291     106
224747     196
268890     101
300421     185
344968     205
351997     144
450083     115
507109     101
620405     143
863370     107
888699     102
913694     133
958080     152
1028618    105
1093518    140
1167872    110
1225517    106
1478258    113
1533234    107
2253724    108
2325790    102
Name: order_id, dtype: int64

### A person with ID 344968 made more (205) purchases than anyone else

In [13]:
orders_per_customers[orders_per_customers == orders_per_customers.max()]  #there is a record

user_id
344968    205
Name: order_id, dtype: int64

In [14]:
print('number of retailers is : ',orders_c.retailer.nunique())

number of retailers is :  61


### There is 61 different retailer in our data. We can analyze how people bahave in different retailers

In [15]:
retailer_total = orders_c.groupby(['retailer'])['order_id'].count()
#here we have number of purshases in different retailers
retailer_total

retailer
220 ВОЛЬТ             33
BILLA               2735
Gipfel                11
Home Market           17
METRO            1369726
                  ...   
Улыбка Радуги        122
Фреш25               132
Хороший              546
Шан                   59
ЮгОпт                 14
Name: order_id, Length: 61, dtype: int64

### Let's find 3 most popular retailers for customers

In [16]:
retailer_total[retailer_total == retailer_total.max()] 

retailer
METRO    1369726
Name: order_id, dtype: int64

### So top 1 retailer is METRO

In [17]:
retailer_total[retailer_total > 100000 ] 

retailer
METRO    1369726
Ашан      401004
Лента     653387
Name: order_id, dtype: int64

### TOP 3 retailers are : Metro, Ашан, Лента 

### bacically product markets are more popular than any other retailers

In [18]:
print('number of platforms is : ',orders_c.platform.nunique())

number of platforms is :  3


### **We have only 3 platforms so we can analyze how many customers made their purchases there**

In [19]:
orders_c.groupby(['platform'])['user_id'].count()

platform
app          1872404
undefined      76536
web           513018
Name: user_id, dtype: int64

### **We can clearly see that purhases in the app are more preferable than purchases in web, also there is 76536 rows with underfined platform **

//------------------------------------------------------------------------------------------------------------------------//

### Let's analyze products dataset

In [20]:
products = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/sbermarket_tab_2_1/tab_2_products_2020-01-01.csv')
products.sample(5)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id
204339,553653,5508213,28986597,103.000000,1,0.000000,Рис Мистраль Кубань круглозерный белый 900 г,66843,Мистраль,580.0,9
278805,712744,5601798,28868664,94.900002,1,5.000000,Семечки подсолнечные От Мартина Отборные жарен...,58715,От Мартина,97.0,94
214551,617342,5451597,29231792,165.520004,1,0.000000,Перец оранжевый,94301,Без бренда,85.0,84
163996,365012,5265273,28947765,43.189999,1,0.000000,Горох Fine Life колотый 450 г,15427,Fine Life,580.0,9
169888,380175,5155477,28695934,24.990000,1,23.700001,Паштет Hame Деликатесный из мяса птицы,64416,Hame,46.0,42


### **there are some outliers in discount column (negative values) let's filter outliers (negative discount will be equal to zero)**

In [21]:
# products['discount_real'] = (products.discount >= 0)

# products[products.discount_real == False].discount == 0
products[ (products.discount < 0)].discount = 0


C:\Users\zvere\anaconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [22]:
products

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id
0,525,5354800,28381452,116.000000,4,0.000000,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99
1,525,5354800,29242911,56.990002,2,13.080000,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51
2,525,5354800,29242919,64.989998,2,40.020000,Тортилья Delicados оригинальная пшеничная,22107,Delicados,596.0,51
3,525,5354800,29243785,44.990002,6,0.000000,Вода питьевая минеральная BonAqua газированная...,2530,BonAqua,77.0,74
4,525,5354800,29244246,65.900002,6,30.990000,"Газированный напиток Mirinda Апельсин 1,5 л",3818486,Mirinda,76.0,74
...,...,...,...,...,...,...,...,...,...,...,...
303662,725964,5652846,29327078,52.639999,2,23.780001,Черная смородина Fine life быстрозамороженная,94654,Fine Life,592.0,90
303663,725964,5652846,29327108,29.459999,1,39.400002,Помидоры черри красные 250 г,687,Без бренда,85.0,84
303664,725964,5652846,29327109,29.480000,1,29.580000,Помидоры черри сливовидные 250 г,4830888,Без бренда,85.0,84
303665,725964,5652846,29327192,83.220001,1,21.209999,Сыр Bonfesto Моцарелла mini 45% 100 г бзмж,26739,Bonfesto,38.0,35


In [23]:
products.shape

(303667, 11)

In [24]:
products.line_item_id.nunique()

303667

In [25]:
products.order_id.nunique()

12154

In [26]:
products.user_id.nunique()

10580

### Let's analyze how many items in every order and calculate the price for every order
### but first we need to create a new column **cost_real** what will include the price ( taking the discount into account) multiplied by quantity 

In [27]:
products['cost_real'] = (products.price - products.discount) * products.quantity
products.head(3)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
0,525,5354800,28381452,116.000000,4,0.00,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99,464.000000
1,525,5354800,29242911,56.990002,2,13.08,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51,87.820004
2,525,5354800,29242919,64.989998,2,40.02,Тортилья Delicados оригинальная пшеничная,22107,Delicados,596.0,51,49.939995


### Some additional info about products dataset

In [28]:
print('the number of unique products names is', products.product_name.nunique())

the number of unique products names is 26763


In [29]:
print('the number of unique customers is', products.user_id.nunique())

the number of unique customers is 10580


### **Now when we know the number of products and cost we can determine the most expensive products and it's price**

In [30]:
products[products.cost_real == products.cost_real.max()] ## the most expensive 

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
45062,137924,5631897,29129595,159962.046875,1,39.490002,Голень индейки Индилайт без кости охлажденная ...,3818829,Индилайт,122.0,119,159922.556873


In [31]:
most_expensive_product_cost = products.groupby(['product_name'])['price'].max().max()
most_expensive_product_cost

159962.046875

### **We can determine the most expensive products and we can make predictions that people who can buy such an espensive products will buy something in the same field with the same price**

In [32]:
products[(products.price < 100000) & (products.price > 1000)].head(3) #expensive products with high price

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real
355,3225,5350970,28436606,1189.010010,1,510.0,Креветки Бухта изобилия королевская 50/70,5368,Agama,118.0,112,679.010010
453,3674,5307004,28931590,1923.979980,1,0.0,Глазной мускул говяжий Заречное охлажденный ~2...,5214742,Заречное,120.0,119,1923.979980
454,3674,5307004,28932555,2150.189941,1,0.0,Лопатка говяжья Мираторг без кости замороженна...,5063231,Мираторг,121.0,119,2150.189941


### **ALso we have noticed that there are some purchases with quantity more than 1, that means that some people buy the same product more than once. We can analyze it and it will help our model to make predictions for these people better : because if they buy this particular product often there is more chance that they will buy it again**

In [33]:
repeat_customers = products[products.quantity >10]

products['repeated'] = (products.quantity >= 10)
products.groupby(['repeated'])['user_id'].count()

repeated
False    297949
True       5718
Name: user_id, dtype: int64

### **We need to create a column with a number of purshaces for every customer**

In [34]:
number_purchases = products.groupby(['user_id'])['order_id'].count()
products1 = pd.merge(products,number_purchases,on = 'user_id')
products1 = products1.rename(columns = {'order_id_y':'num_purchases', 'order_id_x' :'order_id'})
products = products1
products.num_purchases.nunique()


147

### **There are some columns with quantity = 1 but still with high cost_real. That means that these people buy something in huge portion for once, probably it is people who have their oun markets or cafe. Let't determine them**

In [35]:
opt_customers = products[(products.cost_real > 5000) & (products.quantity < 5)] 
opt_customers.head(2)


,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases
9653,36923,5559719,28574203,2700.0,2,0.0,"Кастрюля Aro 13,5 л",81498,Aro,292.0,291,5400.0,False,35
10450,38943,5471510,28421789,6000.0,1,0.0,Набор посуды Tefal Intuition нержавеющая сталь...,92961,Tefal,334.0,321,6000.0,False,31


### We found 38 wholesalers

### If we think about purchases with little quatity and high cost it is possible that these people bought something big and exensive like technics (fridge,TV, etc). Let's create additional bool column for such people

In [36]:
products['technics'] = (products.cost_real > 5000) & (products.num_purchases < 5) 
products.groupby(['technics'])['user_id'].count()

technics
False    303647
True         20
Name: user_id, dtype: int64

###  **There are a lot of people who are making their purchases according to the discount. We have decided to find out all people who prefer items with discount. Let's add another bool column "Prefer_Discounts"** 

In [37]:
discounts_sum = products.groupby(['user_id'],as_index = False)['discount'].sum()
discounts1= pd.merge(products,discounts_sum,on = 'user_id')
discounts1 = discounts1.rename(columns = {'discount_x':'discount', 'discount_y' :'discount_total'})
products = discounts1


In [38]:
products.head(2)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases,technics,discount_total
0,525,5354800,28381452,116.000000,4,0.00,Лазанья Золотой Петушок мясной слоеный пирог з...,37548,Золотой Петушок,101.0,99,464.000000,False,14,False,630.080004
1,525,5354800,29242911,56.990002,2,13.08,Хлеб Хлебный Дом Геркулес зерновой с отрубями ...,5636,Хлебный Дом,596.0,51,87.820004,False,14,False,630.080004


In [39]:
products['prefer_discounts'] = (products.discount_total > products.discount_total.mean())

In [40]:
products.prefer_discounts.value_counts()

False    199140
True     104527
Name: prefer_discounts, dtype: int64

In [41]:
products.sample(2)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases,technics,discount_total,prefer_discounts
193159,479237,5408390,28839695,86.0,1,0.0,Яйцо перепелиное Qegg для детского питания 20 шт,14844,Qegg,34.0,20,86.0,False,24,False,94.060001,False
145236,340235,5636406,29225363,249.0,1,0.0,Колбаса Велком Краковская полукопченая 450 г,31491,Велком,44.0,42,249.0,False,48,False,1540.549995,True


### **There are also families who have big check, they buy a lot of products for one time. We can find them. Now when we have number of items per user we can detect a families. They buy a lot of products for once so we can group them according do order_id and high number of items in order**

### Also there is an idea to separate customers into three groups : 1,2,3 level of earning according to people's purchases cost and suggest a person to buy some products from separate price intervals

That feature is in our plans for future(more powerful) predictions

### **There are also some favourite products for each individual. It is more possible that person buy something from the list of favourites. Let's find the list of the most popular products**

In [42]:
popular_product_id = products.groupby(['product_id'],as_index = False)['user_id'].count()
popular_product_id = sorted(popular_product_id.values[:,0], reverse=True)
popular_product_id[50]

9152094

In [43]:
products.product_name[products.product_id == popular_product_id[50]].values

array(['Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт',
       'Сырок творожный глазированный Б.Ю. Александров 25 г х 6 шт'],
      dtype=object)

In [44]:
popular_products_names = []
for i in range(50):
    popular_products_names.append(products.product_name[products.product_id == popular_product_id[i]].values[0])

### Before we start with categories let's take a look on how our products dataset is looking now

In [45]:
products.sample(3)

,user_id,order_id,line_item_id,price,quantity,discount,product_name,product_id,brand_name,master_category_id,parent_category_id,cost_real,repeated,num_purchases,technics,discount_total,prefer_discounts
77171,218423,5394922,27547079,219.000000,1,90.0,"Кокосовый напиток Alpro с рисом 0,9% 1 л",20072,Alpro,23.0,20,129.000000,False,66,False,1793.969983,True
91356,252749,5467498,28916266,145.000000,1,0.0,Голубика Fine Life быстрозамороженная 300 г,4831077,Fine Life,592.0,90,145.000000,False,53,False,536.890003,False
255314,701421,5322661,28129947,92.199997,1,0.0,Фасоль Bonduelle красная,1749,Bonduelle,625.0,67,92.199997,False,92,False,957.169991,False


//------------------------------------------------------------------------------------------------------------------------//

### Let's analyze the rest of datasets

### Categories

In [62]:
categories= pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_3_categories.csv')
categories.sample(5)



,id,name,parent_id
77,78,Чай холодный,74
366,367,Маркеры и фломастеры,365
981,982,Батуты,554
378,379,Канцелярские ножницы и ножи,377
619,620,Сыры с плесенью,480


In [64]:
categories.nunique()

id           1016
name          947
parent_id     140
dtype: int64

### We have too many different products to make clusters from it but we can make clusters according to category (1016 clusters) and build our model on it

//------------------------------------------------------------------------------------------------------------------------//

### User profiles

In [47]:
user_profiles = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_4_user_profiles.csv')
user_profiles.sample(5)


,user_id,gender,bdate
438492,3423006,female,1988-05-07
35844,855184,NaN,NaN
10599,2255845,NaN,NaN
308105,3137672,female,1995-01-09
142468,2701572,NaN,NaN


### There are some NaN values in the data. Let's take a look and calculate the number of NaNs in gender and bdate columns

In [48]:
user_profiles.gender.isna().value_counts()

True     231737
False    207130
Name: gender, dtype: int64

In [49]:
user_profiles.bdate.isna().value_counts()

True     231880
False    206987
Name: bdate, dtype: int64

### There is more than a half of people who did not mentioned their info. That is not so informative dataset, but we can take a look on people who mentioned their info and they will probably spend more time reading and fulfilling something in the app than people who did not provide anything

//------------------------------------------------------------------------------------------------------------------------//

### Product properties

In [50]:
product_properties = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/kaggle_tab_1345/tab_5_product_properties.csv')
product_properties.sample(3)                              

,product_id,property_name,property_value
1754792,6784920,Совместимость с кухонной техникой,Посудомоечные машины
1804843,6827453,Состав,"Свинина, вода, соль поваренная пищевая, пищева..."
1168613,5075765,Вид молока,Коровье


### Using the data from product properties dataset we can build strong and complicated model which will analyze property value (probably after making a clusters of products). That idea is also for future developments

### From that point feature engeneering is stopped. There also a lot of good ideas that could be implemented in future but we need to focus on making first predictions using already existing features

//------------------------------------------------------------------------------------------------------------------------//

### Submission data

In [52]:
df = pd.read_csv(r'C:/Users/zvere/Sber/test-recsys/sample_submission.csv')
df.head(3)

,Id,Predicted
0,51,3239211 10720024 8493844 1965540 5009002 95218...
1,65,11693356 13392267 2464515 2245175 4748570 1134...
2,766,12773054 3692749 12726751 10041624 11893515 23...


In [53]:
#split the data
from sklearn.model_selection import train_test_split

In [54]:
numerical = ['price', 'quantity', 'discount','cost_real', 'num_purchases']
categorical = ['user_id', 'order_id', 'product_id', 'repeated']

In [55]:
y = df['Predicted']
X = df['Id']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5,random_state = 7)

### The simpliest idea for a model is to choose 50 most popular products and predict that person will buy his products from that list

In [57]:
popular_id = popular_product_id[:50]
popular_list = []
popular_id[:5]

[13550713, 13550387, 13550375, 13550359, 13550321]

In [58]:
df['my_pred'] = df.Predicted
for i in range(len(df)):
    df.my_pred[i] = popular_id
df.drop('Predicted', axis=1, inplace=True)
df = df.rename(columns = {'my_pred':'Predicted'} )

<ipython-input-58-872096aadfb2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.my_pred[i] = popular_id


In [59]:
df.to_csv('C:/Users/zvere/Sber/test-recsys/submission1.csv', index = False)

In [60]:
cols = ['Id', 'Predicted']
for col in cols:
    df[col] = df[col].map(lambda x :str(x))
df

,Id,Predicted
0,51,"[13550713, 13550387, 13550375, 13550359, 13550..."
1,65,"[13550713, 13550387, 13550375, 13550359, 13550..."
2,766,"[13550713, 13550387, 13550375, 13550359, 13550..."
3,1132,"[13550713, 13550387, 13550375, 13550359, 13550..."
4,1578,"[13550713, 13550387, 13550375, 13550359, 13550..."
...,...,...
107063,2997849,"[13550713, 13550387, 13550375, 13550359, 13550..."
107064,2997853,"[13550713, 13550387, 13550375, 13550359, 13550..."
107065,2997873,"[13550713, 13550387, 13550375, 13550359, 13550..."
107066,2997988,"[13550713, 13550387, 13550375, 13550359, 13550..."


### More complicated idea is to find the purchases person already made (and probably will buy it again), find out if a person prefer discounted products and mix products with discounts and from the past purschases + add some popular products

### Also in our plans we analyze the retailer where person usually buy his purchases and collaborate all ideas from the model described above taking the desided retailer into account